In [1]:
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from anndata import AnnData
from squidpy.pl._utils import save_fig
from time import process_time

sc.logging.print_header()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3
sc.settings.dpi = 300
sq.__version__
sc.settings.figdir = "./figures"

%load_ext autoreload
%autoreload 2
%load_ext lab_black

scanpy==1.7.0rc2.dev1+g2a123065 anndata==0.7.4 umap==0.4.6 numpy==1.19.4 scipy==1.5.2 pandas==1.1.4 scikit-learn==0.23.2 statsmodels==0.12.1 python-igraph==0.8.3 leidenalg==0.8.2


In [2]:
adata_visium = sq.datasets.visium_fluo_adata()
adata_seqfish = sq.datasets.seqfish()
adata_fouri = sq.datasets.four_i()
adata_imc = sq.datasets.imc()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [3]:
from scipy import sparse

adata_visium.uns = {}
adata_fouri.uns = {}
adata_fouri_r = AnnData(
    sparse.csr_matrix(np.empty(adata_fouri.shape)),
    obs=adata_fouri.obs[["cluster"]].copy(),
)
adata_fouri_r.obsm["spatial"] = adata_fouri.obsm["spatial"]
adata_seqfish.obs["cluster"] = adata_seqfish.obs.celltype_mapped_refined
adata_imc.obs["cluster"] = adata_imc.obs["cell type"]

In [4]:
adata_visium.write("/Users/giovanni.palla/.cache/squidpy/visium_R.h5ad")
adata_fouri_r.write("/Users/giovanni.palla/.cache/squidpy/fouri_R.h5ad")
adata_imc.write("/Users/giovanni.palla/.cache/squidpy/imc_R.h5ad")
adata_seqfish.write("/Users/giovanni.palla/.cache/squidpy/seqfish_R.h5ad")

/Users/giovanni.palla/miniconda3/envs/spatial/lib/python3.8/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])


In [ ]:
library(zellkonverter)
library(Giotto)
library(microbenchmark)

sce_seqfish  <- readH5AD("/Users/giovanni.palla/.cache/squidpy/seqfish_R.h5ad")
sce_imc  <- readH5AD("/Users/giovanni.palla/.cache/squidpy/imc_R.h5ad")
sce_visium  <- readH5AD("/Users/giovanni.palla/.cache/squidpy/visium_R.h5ad")
sce_4i  <- readH5AD("/Users/giovanni.palla/.cache/squidpy/fouri_R.h5ad")

data_list <- c(sce_seqfish,sce_imc , sce_visium) #, sce_4i)
id_list <- c("seqfish","imc","visium") #,"4i")

result <- mapply(bench_graph, data_list, id_list, SIMPLIFY=F)

bench_graph <- function(sce, id){
  print(id)
  return(id)
}

for (d in c(c(sce_visium, sce_imc), c("visium","imc"))){
  print(d)
  sce <- data[0]
  dataset_id <- data[1]
  print(dataset_id)
}

gobject_seqfish <- createGiottoObject(raw_exprs = sce@assays@data$X, 
                                      spatial_locs = sce@int_colData$reducedDims$spatial,
                                      cell_metadata = sce@colData$`cell type`)

results_network <- microbenchmark(
  net_delaunay = createSpatialNetwork(gobject,name = "spatial_network",method = 'Delaunay',),
  net_knn  = createSpatialNetwork(gobject,name = "spatial_network",method = 'kNN',),
  unit = "s",
  times = 10)

gobject <- createSpatialNetwork(gobject,
                                name = "spatial_network",
                                method = 'Delaunay',)

results_nhood <- microbenchmark(
  cellproxy = cellProximityEnrichment(
    gobject,
    cluster_column = 'V1',
    spatial_network_name = 'spatial_network',
    adjust_method = 'fdr',
    number_of_simulations = 1000
  ),
  unit = "s",
  times = 10)

df <- rbind(summary(results_network),summary(results_nhoo))
df$dataset = "dataset"
